In [ ]:
!pip install easyocr
!pip install -U tensorflow>=2.5
!git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
%%bash
sudo apt install -y protobuf-compiler
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [4]:
# Importing necessary modules
import os
import cv2
from matplotlib import pyplot as plt
import numpy as np
import imutils
import easyocr
import tensorflow as tf
import tensorflow_hub as hub
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

%matplotlib inline

In [ ]:
# Using a ResNet50 to detect buses
model_display_name = "Faster R-CNN ResNet50 V1 640x640"
hub_model = hub.load('https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1')

In [ ]:
# Defining labels
PATH_TO_LABELS = './models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [ ]:
# Reshaping the image for the model and loading it
def reshape_image(image_file):
    image_data = tf.io.gfile.GFile(image_file, 'rb').read()
    image = Image.open(BytesIO(image_data))
    (image_width, image_height) = image.size

    img = np.array(image.getdata()).reshape((1, image_height, image_width, 3)).astype(np.uint8)
    img[0] = np.tile(np.mean(img[0], 2, keepdims=True), (1,1,3)).astype(np.uint8)
    return img

def get_results(image):
    results = hub_model(image)
    result = {key: value.numpy() for key,value in results.items()}

    return result

In [ ]:
# Displaying the labels detected
def display_labels(image, result):
    label_id_offset = 0
    image_np_with_detections = None
    image_np_with_detections = image.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections[0],
      result['detection_boxes'][0],
      (result['detection_classes'][0] + label_id_offset).astype(int),
      result['detection_scores'][0],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.30,
      agnostic_mode=False)

    plt.figure(figsize=(24,32))
    plt.imshow(image_np_with_detections[0])
    plt.show()

In [ ]:
# Cropping the original image with the bus box
def crop_image(image, box):
    imag = cv2.imread(image)
    orgShape = imag.shape
    
    reShape = (640, 640)
    xVal = orgShape[0] / reShape[0]
    yVal = orgShape[1] / reShape[1]

    if isinstance(box[0], np.ndarray):
        box = box[0]

    box[0] = (box[0] * xVal)
    box[2] = (box[2] * xVal)
    box[1] = (box[1] * yVal)
    box[3] = (box[3] * yVal)

    # plt.imshow(cv2.cvtColor(imag, cv2.COLOR_BGR2RGB))
    imag = cv2.cvtColor(imag, cv2.COLOR_BGR2RGB)
    final = imag[box[0] : box[2], box[1] : box[3]]
    
    # plt.figure()

    #subplot(r,c) provide the no. of rows and columns
    # figure, axis= plt.subplots(2,1) 

    # use the created array to output your multiple images. In this case I have stacked 4 images vertically
    # axis[0].set_title("Before")
    # axis[0].imshow(imag)
    
    # axis[1].set_title("After")
    # axis[1].imshow(final)

    return final   

In [ ]:
# Converting the image to grayscale
def convert_grayscale(image):
    gray = cv2.csvColor(image, cv2.COLOR_BGR2GRAY)
    cv2.imwrite("grayscaled.jpg", gray)
    
    return gray

In [ ]:
# Using EasyOCR to detect text boxes
def read_display_with_EasyOCR(image):
    reader = easyocr.Reader(['pt', 'en'])
    res = reader.detect(image)
    print(res)

    top_left = (res[0][2][0], res[0][2][2])
    bottom_right = (res[0][2][1], res[0][2][3])
    
    final = cv2.imread(image)
    final = cv2.rectangle(final, top_left, bottom_right, (0, 255, 0), 5)
    
    plt.imshow(final)

    return final

In [ ]:
def createGray(filename):
    # Converting the image to grayscale
    final = cv2.imread(filename)
    gray = cv2.cvtColor(final, cv2.COLOR_BGR2GRAY)
    return gray

In [ ]:
def readText(filename):
    # Using EasyOCR to detect text boxes
    reader = easyocr.Reader(['pt', 'en'], recognizer='Transformer')
    res = reader.readtext(filename)
    return res

In [ ]:
# Returns the similarity between two strings
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
# Returns 'Valter Silva' text box coordinates
def getValter(filename, res):
    valterBox = []
    for i in res:
        if similar("Valter Silva", i[1]) > 0.8:
            print(i[1])
            valterBox.append(i[0][0])
            valterBox.append(i[0][2])
            valterBox = [item for sublist in valterBox for item in sublist]
            return valterBox

In [ ]:
# Adds a black rectangle on 'Valter Silva' text
def removeValter(filenameOpen, valterBox):
    imag = cv2.imread(filenameOpen)
    imag = cv2.rectangle(imag, (valterBox[0], valterBox[1]), (valterBox[2], valterBox[3]), (0, 0, 0), -1)
    return imag

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Looping through all the images to crop the bus and 'Valter Silva' text
path = "/content/drive/MyDrive/ProjetoBUS/Clean/"
grayPath = "/content/drive/MyDrive/ProjetoBUS/DatasetCinza/"
cleanPath = "/content/drive/MyDrive/ProjetoBUS/DatasetFinal/"


for i in range(6327):
    print(i)
    if not os.path.isfile(f"{path}/img{i}.jpg"):
        continue
    if os.path.isfile(f"{cleanPath}/img{i}.jpg"):
        continue

    image = reshape_image(f"{path}/img{i}.jpg")
    result = get_results(image)

    busDetection = result['detection_classes'] == 6
    scoreDetection = result['detection_scores'] > 0.95
    box = result['detection_boxes'][np.where(busDetection & scoreDetection)]
    box = box.squeeze()
    box *= 640
    box = box.astype(np.int32)
    if len(box) == 0:
        continue
    cv2.imwrite(f"{grayPath}/img{i}.jpg", crop_image(f"{path}/img{i}.jpg", box))
    # cv2.cvtColor(cv2.imread(f"{path}/img{i}.jpg"), cv2.COLOR_BGR2LAB)
    cv2.imwrite(f"{grayPath}/img{i}.jpg", createGray(f"{grayPath}/img{i}.jpg"))
    res = readText(f"{grayPath}/img{i}.jpg")
    valterBox = getValter(f"{grayPath}/img{i}.jpg", res)
    if valterBox == None:
        continue
    valterBox = [int(item) for item in valterBox]
    print(valterBox)
    cv2.imwrite(f"{cleanPath}/img{i}.jpg", removeValter(f"{grayPath}/img{i}.jpg", valterBox))